In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from flask import Flask, render_template, request
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
app = Flask(__name__)

In [3]:
def get_model():
    global model
    model = load_model('model.h5')
    print("Model loaded!")

In [4]:
def load_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    return img_tensor

In [5]:
def prediction(img_path):
    new_image = load_image(img_path)
    
    pred = model.predict(new_image)
    
    print(pred)
    
    labels=np.array(pred)
    labels[labels>=0.6]=1
    labels[labels<0.6]=0
    
    print(labels)
    final=np.array(labels)
    
    if final[0][0]==1:
        return "Bad"
    else:
        return "Good"

In [6]:
get_model()

Model loaded!


In [7]:
@app.route("/", methods=['GET', 'POST'])
def home():

    return render_template('home.html')

@app.route("/predict", methods = ['GET','POST'])
def predict():
    
    if request.method == 'POST':
        
        file = request.files['file']
        filename = file.filename
        file_path = os.path.join(r'static/', filename)                       #slashes should be handeled properly
        file.save(file_path)
        print(filename)
        product = prediction(file_path)
        print(product)
        
    return render_template('predict.html', product = product, user_image = file_path)            #file_path can or may used at the place of filename

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Aug/2022 12:19:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 12:19:28] "GET /favicon.ico HTTP/1.1" 404 -


bad (24)k.jpeg


127.0.0.1 - - [25/Aug/2022 12:19:54] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 12:19:54] "GET /static/bad%20(24)k.jpeg HTTP/1.1" 200 -


[[0.99746203 0.00253799]]
[[1. 0.]]
Bad


127.0.0.1 - - [25/Aug/2022 12:22:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 12:22:21] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 12:22:21] "GET /static/good%20(447)k.jpeg HTTP/1.1" 200 -


good (447)k.jpeg
[[1.2706597e-05 9.9998724e-01]]
[[0. 1.]]
Good


127.0.0.1 - - [25/Aug/2022 12:22:32] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 12:22:32] "GET /static/bad%20(106)k.jpeg HTTP/1.1" 200 -


bad (106)k.jpeg
[[9.9999940e-01 5.8715597e-07]]
[[1. 0.]]
Bad
